In [28]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input
import os

In [15]:
from keras.layers import Dropout
from tensorflow.keras.utils import plot_model 

In [3]:
classes = ['самолет', 'автомобиль', 'птица', 'кот', 'олень', 'собака', 'лягушка', 'лошадь', 'корабль', 'грузовик']

In [4]:
batch_size = 512
num_classes = 10
epochs = 5
data_augmentation = False
num_predictions = 20

# разделение тренировочной и тестовой выборки
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'тренировочные примеры')
print(X_test.shape[0], 'тестовые примеры')

170508288/170498071 [==============================] - 455s 3us/step
X_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


In [6]:
# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


X_train = X_train / 255.0
X_test = X_test / 255.0

### Baseline. 2 свёртки, пуллинг

In [14]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


In [18]:
# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 activation_14 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 conv2d_11 (Conv2D)          (None, 30, 30, 32)        9248      
                                                                 
 activation_15 (Activation)  (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 15, 15, 32)        0         
                                                      

In [19]:
data_augmentation = True
if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=10, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0.2, 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)


    train_gen = datagen.flow(X_train, 
                             y_train,
                             batch_size=batch_size)

    # запуск data augmentation через fit_generator
    model.fit(train_gen,
                        epochs=epochs,
                        validation_data=(X_test, y_test),
                        workers=4)

Использование data augmentation
Epoch 1/5
98/98 [==============================] - 63s 629ms/step - loss: 2.3030 - accuracy: 0.1001 - val_loss: 2.3027 - val_accuracy: 0.1000
Epoch 2/5
98/98 [==============================] - 66s 668ms/step - loss: 2.3029 - accuracy: 0.0996 - val_loss: 2.3027 - val_accuracy: 0.1000
Epoch 3/5
98/98 [==============================] - 71s 721ms/step - loss: 2.3027 - accuracy: 0.1015 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 4/5
98/98 [==============================] - 78s 784ms/step - loss: 2.3029 - accuracy: 0.1010 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 5/5
98/98 [==============================] - 77s 781ms/step - loss: 2.3028 - accuracy: 0.0996 - val_loss: 2.3026 - val_accuracy: 0.1000


### Увеличение слоёв conv-pool, изменение оптимизатора на ADAM

In [20]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='ADAM',
              metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 activation_22 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 conv2d_13 (Conv2D)          (None, 30, 30, 32)        9248      
                                                                 
 activation_23 (Activation)  (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 15, 15, 32)        0         
                                                      

In [21]:
data_augmentation = True
if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=10, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0.2, 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)


    train_gen = datagen.flow(X_train, 
                             y_train,
                             batch_size=batch_size)

    # запуск data augmentation через fit_generator
    model.fit(train_gen,
                        epochs=epochs,
                        validation_data=(X_test, y_test),
                        workers=4)

Использование data augmentation
Epoch 1/5
98/98 [==============================] - 92s 931ms/step - loss: 1.8358 - accuracy: 0.3201 - val_loss: 1.5154 - val_accuracy: 0.4545
Epoch 2/5
98/98 [==============================] - 107s 1s/step - loss: 1.5315 - accuracy: 0.4415 - val_loss: 1.3589 - val_accuracy: 0.5065
Epoch 3/5
98/98 [==============================] - 105s 1s/step - loss: 1.3994 - accuracy: 0.4944 - val_loss: 1.2245 - val_accuracy: 0.5571
Epoch 4/5
98/98 [==============================] - 110s 1s/step - loss: 1.3006 - accuracy: 0.5343 - val_loss: 1.1329 - val_accuracy: 0.5925
Epoch 5/5
98/98 [==============================] - 112s 1s/step - loss: 1.2281 - accuracy: 0.5600 - val_loss: 1.0359 - val_accuracy: 0.6334


In [31]:
# перепишу модель с разделением на две ветки
import tensorflow as tf


first_input = Input(shape=(X_train.shape[1:])) #(32, 32, 3)
x11= Conv2D(32, (3, 3),activation='relu',padding = 'same')(first_input)
x11= Conv2D(32, (3, 3),activation='relu',padding = 'same')(x11)
x11= MaxPooling2D(pool_size=(2, 2))(x11)
# x11= Flatten()(x11)

x12= Conv2D(16, (3, 3),activation='relu',padding = 'same')(first_input)
x12= Conv2D(16, (3, 3),activation='relu',padding = 'same')(x12)
x12= MaxPooling2D(pool_size=(2, 2))(x12)
# x12= Flatten()(x11)

merge_one = Concatenate(  axis =-1 )([x11, x12])

first_dense= Flatten()(merge_one)
first_dense=Dense(256, activation='relu')(first_dense)
first_dense=Dense(num_classes, activation='softmax')(first_dense)

model_stek = Model(inputs=first_input, outputs=first_dense)
ada_grad = tf.keras.optimizers.Adagrad(lr=0.1, epsilon=1e-08, decay=0.0)
sgd = tf.keras.optimizers.SGD()
model_stek.compile(optimizer=sgd, loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
model_stek.fit(X_train, y_train, epochs = 5, validation_split=0.2)

Epoch 1/5
1250/1250 [==============================] - 92s 73ms/step - loss: 1.8667 - accuracy: 0.3352 - val_loss: 1.6462 - val_accuracy: 0.4136
Epoch 2/5
1250/1250 [==============================] - 94s 75ms/step - loss: 1.5247 - accuracy: 0.4601 - val_loss: 1.4386 - val_accuracy: 0.4905
Epoch 3/5
1250/1250 [==============================] - 95s 76ms/step - loss: 1.3493 - accuracy: 0.5254 - val_loss: 1.3332 - val_accuracy: 0.5296
Epoch 4/5
1250/1250 [==============================] - 100s 80ms/step - loss: 1.2096 - accuracy: 0.5727 - val_loss: 1.1933 - val_accuracy: 0.5813
Epoch 5/5
1250/1250 [==============================] - 98s 78ms/step - loss: 1.0900 - accuracy: 0.6173 - val_loss: 1.1148 - val_accuracy: 0.6080


### Вывод
В целом, увеличение слоёв и сложности модели приводит к улучшению метрик,
но значительно увеличивается время обучения
Не стоит бездумно расширять модель: вторая модель с оптимизатором ADAM показала метрики лучше, чем последняя модель из двух веток
    